# CS506 Project — Food Insecurity & Chronic Disease
### Heatmaps-Only Notebook

This notebook recreates the county-level heatmaps used in the midterm report and adds several new stratified visualizations. It:
- Loads raw data from Feeding America (MMG), USDA Food Access Research Atlas, and CDC PLACES
- Processes data to the county level using FIPS codes
- Generates nine county-level choropleth heatmaps using Plotly Express.

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

COUNTY_GEOJSON_URL = "https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json"

## 1. Feeding America (MMG) — Food Insecurity Data

In [2]:
# Load Feeding America Map the Meal Gap data (2015)
mmg = pd.read_excel('./data/raw/feeding_america/MMG2017_2015.xlsx')

# Ensure FIPS is a 5-character string
mmg['FIPS'] = mmg['FIPS'].astype(str).str.zfill(5)

mmg.head()

,FIPS,State,"County, State",2015 Food Insecurity Rate,# of Food Insecure Persons in 2015,Low Threshold in state,Low Threshold Type,High Threshold in state,High Threshold Type,% FI ≤ Low Threshold,% FI Btwn Thresholds,% FI > High Threshold,2015 Child food insecurity rate,# of Food Insecure Children in 2015,% food insecure children in HH w/ HH incomes below 185 FPL in 2015,% food insecure children in HH w/ HH incomes above 185 FPL in 2015,2015 Cost Per Meal,2015 Weighted Annual Food Budget Shortfall
0,01001,AL,"Autauga County, Alabama",0.139,7680,1.3,SNAP,1.85,Other Nutrition Program,0.472,0.221,0.307,0.210,2950,0.73,0.27,3.18,4376000
1,01003,AL,"Baldwin County, Alabama",0.130,25350,1.3,SNAP,1.85,Other Nutrition Program,0.526,0.168,0.306,0.220,9620,0.72,0.28,3.40,15466000
2,01005,AL,"Barbour County, Alabama",0.234,6290,1.3,SNAP,1.85,Other Nutrition Program,0.556,0.236,0.208,0.296,1700,0.80,0.20,3.05,3442000
3,01007,AL,"Bibb County, Alabama",0.161,3650,1.3,SNAP,1.85,Other Nutrition Program,0.538,0.256,0.206,0.250,1230,0.98,0.02,2.89,1888000
4,01009,AL,"Blount County, Alabama",0.113,6510,1.3,SNAP,1.85,Other Nutrition Program,0.640,0.175,0.186,0.239,3260,0.75,0.25,3.02,3528000


## 2. USDA Food Access Research Atlas — Food Access & Demographic Data

In [11]:
## 2. USDA Food Access Research Atlas — Food Access & Demographic Data (2015)

# Load USDA 2015 Atlas dataset (Excel)
atlas = pd.read_excel('./data/raw/usda_food_access/FoodAccessResearchAtlasData2015.xlsx',
                     sheet_name = 2,
                     dtype = 'str')

# Ensure Census Tract column is string and 11 digits
atlas['CensusTract'] = atlas['CensusTract'].astype(str).str.zfill(11)

# Derive county-level FIPS: first 2 digits = state, next 3 digits = county
atlas['StateFIPS'] = atlas['CensusTract'].str[:2]
atlas['CountyFIPS'] = atlas['CensusTract'].str[2:5]
atlas['FIPS'] = atlas['StateFIPS'] + atlas['CountyFIPS']
numeric_cols = [
    'POP2010',
    'lapop1',
    'lalowi1',
    'lakids1',
    'laseniors1',
    'TractKids',
    'TractSeniors'
]

for col in numeric_cols:
    if col in atlas.columns:
        atlas[col] = pd.to_numeric(atlas[col], errors='coerce')

# Now aggregate USDA variables to county level
county_metrics = atlas.groupby('FIPS').agg(
    pop2010=('POP2010', 'sum'),

    # general low-access population (1 mile urban / 10 mile rural)
    la_pop=('lapop1', 'sum'),

    # low-income + low-access population
    la_low_income=('lalowi1', 'sum'),

    # children and senior low-access counts
    kids_low_access=('lakids1', 'sum'),
    total_kids=('TractKids', 'sum'),

    seniors_low_access=('laseniors1', 'sum'),
    total_seniors=('TractSeniors', 'sum')
).reset_index()

# Derived percentages (now safe)
county_metrics['pct_low_access'] = county_metrics['la_pop'] / county_metrics['pop2010']
county_metrics['pct_low_income_low_access'] = county_metrics['la_low_income'] / county_metrics['pop2010']

county_metrics['pct_kids_low_access'] = np.where(
    county_metrics['total_kids'] > 0,
    county_metrics['kids_low_access'] / county_metrics['total_kids'],
    np.nan
)

county_metrics['pct_seniors_low_access'] = np.where(
    county_metrics['total_seniors'] > 0,
    county_metrics['seniors_low_access'] / county_metrics['total_seniors'],
    np.nan
)

county_metrics.head()

,FIPS,pop2010,la_pop,la_low_income,kids_low_access,total_kids,seniors_low_access,total_seniors,pct_low_access,pct_low_income_low_access,pct_kids_low_access,pct_seniors_low_access
0,01001,54571,36469.872893,12899.819468,9721.503804,14613,4245.641526,6546,0.668301,0.236386,0.665264,0.648586
1,01003,182265,131649.158741,44343.516539,30468.848829,41898,21633.863019,30568,0.722295,0.243291,0.727215,0.707729
2,01005,27457,19197.262383,9721.108548,3752.692700,6015,2560.043174,3909,0.699176,0.354048,0.623889,0.654910
3,01007,22915,18918.377818,8937.631915,4252.431906,5201,2286.891978,2906,0.825589,0.390034,0.817618,0.786955
4,01009,57322,52242.222280,20568.194561,12930.509997,14106,7301.222109,8439,0.911382,0.358819,0.916667,0.865176


In [12]:
# Aggregate tract-level Atlas data to county level for key metrics
county_metrics = atlas.groupby('FIPS').agg(
    pop2010=('POP2010', 'sum'),
    la_pop=('lapop1', 'sum'),              # low-access population (1 mile / 10 miles threshold)
    la_low_income=('lalowi1', 'sum'),      # low-income & low-access population
    kids_low_access=('lakids1', 'sum'),
    total_kids=('TractKids', 'sum'),
    seniors_low_access=('laseniors1', 'sum'),
    total_seniors=('TractSeniors', 'sum')
).reset_index()

# Percentage of population with low access to food (generic, any income)
county_metrics['pct_low_access'] = county_metrics['la_pop'] / county_metrics['pop2010']

# Percentage of population that is both low-income and low-access ("food desert" proxy)
county_metrics['pct_low_income_low_access'] = county_metrics['la_low_income'] / county_metrics['pop2010']

# Child and senior low-access percentages (handle division by zero)
county_metrics['pct_kids_low_access'] = np.where(
    county_metrics['total_kids'] > 0,
    county_metrics['kids_low_access'] / county_metrics['total_kids'],
    np.nan,
)
county_metrics['pct_seniors_low_access'] = np.where(
    county_metrics['total_seniors'] > 0,
    county_metrics['seniors_low_access'] / county_metrics['total_seniors'],
    np.nan,
)

county_metrics.head()

,FIPS,pop2010,la_pop,la_low_income,kids_low_access,total_kids,seniors_low_access,total_seniors,pct_low_access,pct_low_income_low_access,pct_kids_low_access,pct_seniors_low_access
0,01001,54571,36469.872893,12899.819468,9721.503804,14613,4245.641526,6546,0.668301,0.236386,0.665264,0.648586
1,01003,182265,131649.158741,44343.516539,30468.848829,41898,21633.863019,30568,0.722295,0.243291,0.727215,0.707729
2,01005,27457,19197.262383,9721.108548,3752.692700,6015,2560.043174,3909,0.699176,0.354048,0.623889,0.654910
3,01007,22915,18918.377818,8937.631915,4252.431906,5201,2286.891978,2906,0.825589,0.390034,0.817618,0.786955
4,01009,57322,52242.222280,20568.194561,12930.509997,14106,7301.222109,8439,0.911382,0.358819,0.916667,0.865176


## 3. CDC PLACES — Chronic Disease Outcomes

In [14]:
# Load CDC PLACES county-level data (2020 release)
places = pd.read_csv('./data/raw/cdc_health/PLACES__County_Data_(GIS_Friendly_Format),_2020_release_20251027.csv')

# Create 5-digit FIPS string
places['FIPS'] = places['CountyFIPS'].astype(str).str.zfill(5)

# Chronic illness measures to average
chronic = ['ARTHRITIS', 'BPHIGH', 'CANCER', 'CASTHMA',
           'CHD', 'COPD', 'DIABETES', 'HIGHCHOL',
           'KIDNEY', 'STROKE', 'OBESITY']
crude_cols = [f"{m}_CrudePrev" for m in chronic]

# Average crude prevalence across major chronic diseases
places['Chronic_CrudePrev_Mean'] = places[crude_cols].mean(axis=1)

places[['FIPS', 'Chronic_CrudePrev_Mean']].head()

,FIPS,Chronic_CrudePrev_Mean
0,01001,18.072727
1,01003,17.754545
2,01005,22.027273
3,01007,19.309091
4,01009,19.181818


## 4. County-Level Choropleth Heatmaps

### 4.1 Heatmap 1 — % of County Population in a Food Desert

In [29]:
fig = px.choropleth(
    county_metrics,
    geojson=COUNTY_GEOJSON_URL,
    locations='FIPS',
    color='pct_low_income_low_access',
    range_color=(county_metrics['pct_low_income_low_access'].min(),
                 county_metrics['pct_low_income_low_access'].max()),
    scope='usa',
    labels={'pct_low_income_low_access': '% Food Desert Pop'},
    color_continuous_scale=px.colors.sequential.Magma
)

fig.update_layout(
    title_text='% of County Population in a Food Desert',
    title_x=0.5,
    width=1000,
    height=500,
    margin=dict(l=20, r=20, t=60, b=20)
)
fig.show()

### 4.2 Heatmap 2 — Food Insecurity Rate by County (2015)

In [30]:
fig = px.choropleth(
    mmg,
    geojson=COUNTY_GEOJSON_URL,
    locations='FIPS',
    color='2015 Food Insecurity Rate',
    scope='usa',
    labels={'2015 Food Insecurity Rate': 'Food Insecurity Rate'},
    title='Food Insecurity Rate by County (2015)'
)

fig.update_layout(
    title_x=0.5,
    width=1000,
    height=500,
    margin=dict(l=20, r=20, t=60, b=20)
)
fig.show()

### 4.3 Heatmap 3 — Average Chronic Disease Prevalence by County

In [31]:
fig = px.choropleth(
    places,
    geojson=COUNTY_GEOJSON_URL,
    locations='FIPS',
    color='Chronic_CrudePrev_Mean',
    scope='usa',
    labels={'Chronic_CrudePrev_Mean': 'Avg Chronic Disease Prevalence (%)'},
    title='Average Chronic Disease Prevalence by County'
)

fig.update_layout(
    title_x=0.5,
    width=1000,
    height=500,
    margin=dict(l=20, r=20, t=60, b=20)
)
fig.show()

### 4.4 Heatmap 4 — % of County Population with Low Food Access

In [32]:
fig = px.choropleth(
    county_metrics,
    geojson=COUNTY_GEOJSON_URL,
    locations='FIPS',
    color='pct_low_access',
    range_color=(county_metrics['pct_low_access'].min(),
                 county_metrics['pct_low_access'].max()),
    scope='usa',
    labels={'pct_low_access': '% Low Access Pop'},
    color_continuous_scale=px.colors.sequential.Turbo[::-1]
)

fig.update_layout(
    title_text='% of County Population with Low Access to Food (1 mile threshold)',
    title_x=0.5,
    width=1000,
    height=500,
    margin=dict(l=20, r=20, t=60, b=20)
)
fig.show()

### 4.5 Heatmap 5 — Child Food Insecurity Rate by County

In [33]:
fig = px.choropleth(
    mmg,
    geojson=COUNTY_GEOJSON_URL,
    locations='FIPS',
    color='2015 Child food insecurity rate',
    scope='usa',
    labels={'2015 Child food insecurity rate': 'Child Food Insecurity Rate'},
    title='Child Food Insecurity Rate by County (2015)'
)

fig.update_layout(
    title_x=0.5,
    width=1000,
    height=500,
    margin=dict(l=20, r=20, t=60, b=20)
)
fig.show()

### 4.6 Heatmap 6 — % of Food Insecure Children in Low-Income Households

In [34]:
col_name = '% food insecure children in HH w/ HH incomes below 185 FPL in 2015'
fig = px.choropleth(
    mmg,
    geojson=COUNTY_GEOJSON_URL,
    locations='FIPS',
    color=col_name,
    scope='usa',
    labels={col_name: '% FI Children in <185% FPL HHs'},
    title='% of Food Insecure Children in Households <185% FPL (2015)'
)

fig.update_layout(
    title_x=0.5,
    width=1000,
    height=500,
    margin=dict(l=20, r=20, t=60, b=20)
)
fig.show()

### 4.7 Heatmap 7 — % of Children Living in Low-Access Areas

In [35]:
fig = px.choropleth(
    county_metrics,
    geojson=COUNTY_GEOJSON_URL,
    locations='FIPS',
    color='pct_kids_low_access',
    scope='usa',
    labels={'pct_kids_low_access': '% Children Low Access'},
    title='% of Children Living in Low-Access Food Areas (1 mile threshold)'
)

fig.update_layout(
    title_x=0.5,
    width=1000,
    height=500,
    margin=dict(l=20, r=20, t=60, b=20)
)
fig.show()

### 4.8 Heatmap 8 — % of Seniors Living in Low-Access Areas

In [36]:
fig = px.choropleth(
    county_metrics,
    geojson=COUNTY_GEOJSON_URL,
    locations='FIPS',
    color='pct_seniors_low_access',
    scope='usa',
    labels={'pct_seniors_low_access': '% Seniors Low Access'},
    title='% of Seniors Living in Low-Access Food Areas (1 mile threshold)'
)

fig.update_layout(
    title_x=0.5,
    width=1000,
    height=500,
    margin=dict(l=20, r=20, t=60, b=20)
)
fig.show()

### 4.9 Heatmap 9 — Obesity Prevalence by County

In [37]:
fig = px.choropleth(
    places,
    geojson=COUNTY_GEOJSON_URL,
    locations='FIPS',
    color='OBESITY_CrudePrev',
    scope='usa',
    labels={'OBESITY_CrudePrev': 'Obesity Prevalence (%)'},
    title='Obesity Prevalence by County'
)

fig.update_layout(
    title_x=0.5,
    width=1000,
    height=500,
    margin=dict(l=20, r=20, t=60, b=20)
)
fig.show()

### 4.10 Heatmap 10 — Diabetes Prevalence by County

In [38]:
fig = px.choropleth(
    places,
    geojson=COUNTY_GEOJSON_URL,
    locations='FIPS',
    color='DIABETES_CrudePrev',
    scope='usa',
    labels={'DIABETES_CrudePrev': 'Diabetes Prevalence (%)'},
    title='Diabetes Prevalence by County'
)

fig.update_layout(
    title_x=0.5,
    width=1000,
    height=500,
    margin=dict(l=20, r=20, t=60, b=20)
)
fig.show()